In [1]:
import sqlite3
import plyvel
from pathlib import Path
import re
import json
from functions.runHealth import runHealth
from functions.dynamic_analysis import Dynamic

In [2]:
# path : Path = Path('/home/ndanner_plp/OpenWPM/crawl-data/datadir2000')
path : Path = Path('../../datadirTesting')
con : sqlite3.Connection = sqlite3.connect( str(path.joinpath("crawl-data.sqlite")) )
db = plyvel.DB( str(path.joinpath("leveldb")) )
%load_ext sql
%sql sqlite:///{path}/crawl-data.sqlite

'Connected: @../../datadirTesting/crawl-data.sqlite'

In [3]:
n, f = runHealth(con)
print(f"total visits: {n}, failed/incomplete visits: {f}. Success percentage: {round(100* (1 - f/n)) }%")

total visits: 6, failed/incomplete visits: 0. Success percentage: 100%


In [ ]:
for key, value in db:
    print(key)
    print(value, end="\n\n")

In [5]:
results = Dynamic(con)
print( json.dumps( results, indent= 4 ) )

Done: 0 out of 360
Found: (2244797484084519, 'https://fingerprint-react-demo.vercel.app/static/js/main.88a33984.js')
Found: (2880884688549523, '')
Found: (8048464006770836, 'https://fingerprint-react-demo.vercel.app/static/js/main.88a33984.js')
{
    "Canvas": [
        [
            2244797484084519,
            "https://fingerprint-react-demo.vercel.app/static/js/main.88a33984.js"
        ],
        [
            2880884688549523,
            ""
        ],
        [
            8048464006770836,
            "https://fingerprint-react-demo.vercel.app/static/js/main.88a33984.js"
        ]
    ]
}


In [6]:
# %load_ext sql
# symbols = {}
# for path in Path("/home/ndanner_plp/OpenWPM/crawl-data").iterdir():
#     if path.is_dir() and re.match("^datadir" ,path.name):
#         %sql sqlite:///{path}/crawl-data.sqlite
#         result = %sql SELECT DISTINCT(symbol) FROM javascript
#         if result:
#             for symbol in result.DataFrame()['symbol']:
#                 if symbol in symbols:
#                     symbols[symbol].append(path.name)
#                 else:
#                     symbols[symbol] = [path.name]
# print( json.dumps( symbols, indent= 4 ) )

In [3]:
%%sql
SELECT * 
FROM site_visits

 * sqlite:///../../datadirTesting/crawl-data.sqlite
Done.


visit_id,browser_id,site_url,site_rank
434043217340988,1330075446,https://fingerprint-static-demo.vercel.app/public/html/clientjs.html,4
2244797484084519,1330075446,https://fingerprint-react-demo.vercel.app/imprintjs,2
2880884688549523,1330075446,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,3
6261079636704926,1330075446,https://fingerprint-react-demo.vercel.app/clientjs,1
8048464006770836,1330075446,https://fingerprint-react-demo.vercel.app/fingerprintjs,0
8838885414574341,1330075446,https://fingerprint-static-demo.vercel.app/public/html/imprintjs.html,5


In [ ]:
%%sql
SELECT * 
FROM http_responses
WHERE content_hash <> ""
# WHERE visit_id = '{2880884688549523}' 

In [17]:
%%sql
SELECT * 
FROM javascript
WHERE script_url = "" AND visit_id = 2880884688549523

 * sqlite:///../../datadirTesting/crawl-data.sqlite
Done.


id,incognito,browser_id,visit_id,extension_session_uuid,event_ordinal,page_scoped_event_ordinal,window_id,tab_id,frame_id,script_url,script_line,script_col,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp
166,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,6,0,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,isWebKit,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,window.navigator.vendor,get,,None,2022-12-06T03:54:48.639Z
167,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,7,1,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,isChromium,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,window.navigator.vendor,get,,None,2022-12-06T03:54:48.659Z
168,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,8,2,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,isAndroid,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,window.navigator.appVersion,get,5.0 (X11),None,2022-12-06T03:54:48.660Z
169,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,9,3,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,isWebKit,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,window.navigator.vendor,get,,None,2022-12-06T03:54:48.660Z
170,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,10,4,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,getAudioFingerprint,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,OfflineAudioContext.createOscillator,call,,None,2022-12-06T03:54:48.660Z
171,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,11,5,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,getAudioFingerprint,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,OscillatorNode.type,set,triangle,None,2022-12-06T03:54:48.661Z
172,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,12,6,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,getAudioFingerprint,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,OscillatorNode.frequency,get,{},None,2022-12-06T03:54:48.661Z
173,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,13,7,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,getAudioFingerprint,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,OfflineAudioContext.createDynamicsCompressor,call,,None,2022-12-06T03:54:48.661Z
174,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,14,8,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,getAudioFingerprint,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,OscillatorNode.connect,call,,"[""{}""]",2022-12-06T03:54:48.662Z
175,0,1330075446,2880884688549523,31912758-192d-4185-9329-ddbdb75ba73a,15,9,1,1,0,,https,//fingerprint-static-demo.vercel.app/dist/node_modules/,getAudioFingerprint,,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,https://fingerprint-static-demo.vercel.app/public/html/fingerprintjs.html,,OfflineAudioContext.destination,get,{}